# <p class='alert alert-info'>Proyecto final</p>

In [1]:
###
# Importaciones
###
# Data
import pandas as pd
import numpy as np

In [2]:
print("¡Hola, Proyecto Final del Curso!")

¡Hola, Proyecto Final del Curso!
